1. finnhub에서 가져온 뉴스 데이터 확인

In [20]:
read_path = "./TSLA_extended_news_2025-06-14.csv"
write_path = "./tesla_finbert_finnhub.csv"

In [21]:
import pandas as pd

df = pd.read_csv(read_path)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9122 entries, 0 to 9121
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 9122 non-null   int64  
 1   title              9122 non-null   object 
 2   summary            8995 non-null   object 
 3   link               9122 non-null   object 
 4   publisher          0 non-null      float64
 5   category           9122 non-null   object 
 6   pubDate            9122 non-null   object 
 7   image              5022 non-null   object 
 8   related            9122 non-null   object 
 9   source             9122 non-null   object 
 10  collection_period  9122 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 784.1+ KB


In [23]:
df.head(5)

,id,title,summary,link,publisher,category,pubDate,image,related,source,collection_period
0,135270081,Investing.com’s stocks of the week,Investing.com -- The risk environment took a t...,https://finnhub.io/api/news?id=7b50f31ff46e1da...,NaN,company,2025-06-14 09:30:10,https://s.yimg.com/rz/stage/p/yahoo_finance_en...,TSLA,Yahoo,2025-06-07_2025-06-14
1,135270082,How Elon Musk Is Reinventing Tesla’s Strategy,The CEO’s embrace of nascent AI technology is ...,https://finnhub.io/api/news?id=2b8d136bb3816bf...,NaN,company,2025-06-14 09:30:00,https://s.yimg.com/rz/stage/p/yahoo_finance_en...,TSLA,Yahoo,2025-06-07_2025-06-14
2,135271383,YTSL:CA: Second Look Alters My Thesis After 40...,NaN,https://finnhub.io/api/news?id=5ad93e270cefed4...,NaN,company,2025-06-14 09:15:00,NaN,TSLA,SeekingAlpha,2025-06-07_2025-06-14
3,135268235,Harbor Human Capital Factor US Large Cap ETF Q...,"During the first quarter, the Harbor Human Cap...",https://finnhub.io/api/news?id=f9bc3e7c6441a64...,NaN,company,2025-06-14 05:10:00,https://static.seekingalpha.com/cdn/s3/uploads...,TSLA,SeekingAlpha,2025-06-07_2025-06-14
4,135261684,The Real Heirs to Berkshire Hathaway,"Readers weigh in on the mini-Berkshires, small...",https://finnhub.io/api/news?id=108778cee7146af...,NaN,company,2025-06-14 00:23:00,https://s.yimg.com/rz/stage/p/yahoo_finance_en...,TSLA,Yahoo,2025-06-07_2025-06-14


In [24]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

MODEL = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
finbert = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, return_all_scores=True)

df = pd.read_csv(read_path)

df = df.fillna("")

df["text"] = df["title"] + ". " + df["summary"]

def truncate_and_flag(text, tokenizer, max_len=512):
    tokens = tokenizer.encode(text, truncation=True, max_length=max_len)
    truncated_text = tokenizer.decode(tokens, skip_special_tokens=True)
    over_flag = 1 if len(tokenizer.encode(text)) > max_len else 0
    return truncated_text, over_flag

df[["text", "over_512"]] = df["text"].apply(
    lambda x: pd.Series(truncate_and_flag(x, tokenizer, max_len=512))
)

texts = df["text"].tolist()
results = finbert(texts, batch_size=8)

pos_scores, neu_scores, neg_scores = [], [], []
for res in results:
    d = {r["label"].lower(): r["score"] for r in res}
    pos_scores.append(d.get("positive", 0.0))
    neu_scores.append(d.get("neutral", 0.0))
    neg_scores.append(d.get("negative", 0.0))

df["finbert_positive"] = pos_scores
df["finbert_neutral"] = neu_scores
df["finbert_negative"] = neg_scores

df.to_csv(write_path, index=False)
print(f"결과 저장 완료: {write_path}")


Device set to use cuda:0
c:\Users\jhh33\anaconda3\envs\pythonProject1\Lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


결과 저장 완료: ./tesla_finbert_finnhub.csv


Text의 길이가 얼마나 영향을 끼치는지 확인

In [18]:
df = pd.read_csv("./apple_finbert_finnhub.csv")

# 512 토큰 넘은 것만 필터링
df_over = df[df["over_512"] == 1]

# 요약 통계 출력
summary = {
    "총 개수": len(df_over),
    "평균 Positive": df_over["finbert_positive"].mean(),
    "평균 Neutral": df_over["finbert_neutral"].mean(),
    "평균 Negative": df_over["finbert_negative"].mean(),
    "Positive > 0.5": (df_over["finbert_positive"] > 0.5).sum(),
    "Negative > 0.5": (df_over["finbert_negative"] > 0.5).sum(),
    "Neutral > 0.7": (df_over["finbert_neutral"] > 0.7).sum(),
}

for k, v in summary.items():
    print(f"{k}: {v}")


총 개수: 8
평균 Positive: 0.30173408985865535
평균 Neutral: 0.6699639637954533
평균 Negative: 0.028301946480496512
Positive > 0.5: 2
Negative > 0.5: 0
Neutral > 0.7: 5


In [21]:
summary = {
    "총 개수": len(df),
    "평균 Positive": df["finbert_positive"].mean(),
    "평균 Neutral": df["finbert_neutral"].mean(),
    "평균 Negative": df["finbert_negative"].mean(),
    "Positive > 0.5": (df["finbert_positive"] > 0.5).sum(),
    "Negative > 0.5": (df["finbert_negative"] > 0.5).sum(),
    "Neutral > 0.7": (df["finbert_neutral"] > 0.7).sum(),
}

for k, v in summary.items():
    print(f"{k}: {v}")

총 개수: 8769
평균 Positive: 0.2730605699986634
평균 Neutral: 0.5371677767336367
평균 Negative: 0.18977165364321283
Positive > 0.5: 2378
Negative > 0.5: 1650
Neutral > 0.7: 4525
